In [1]:
import cv2
import mediapipe as mp
import csv
import copy
import itertools
import string

In [2]:
mp_draw = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles
mp_hand_detection = mp.solutions.hands

In [3]:
def extract_keypoints(img, hand_landmarks):
    width, height = img.shape[1], img.shape[0]
    keypoints = []

    for _, landmark in enumerate(hand_landmarks.landmark):
        x_pos = min(int(landmark.x * width), width - 1)
        y_pos = min(int(landmark.y * height), height - 1)
        keypoints.append([x_pos, y_pos])

    return keypoints

In [4]:
def normalize_keypoints(keypoints):
    copied_keypoints = copy.deepcopy(keypoints)

    base_x, base_y = 0, 0
    for i, point in enumerate(copied_keypoints):
        if i == 0:
            base_x, base_y = point[0], point[1]
        copied_keypoints[i][0] -= base_x
        copied_keypoints[i][1] -= base_y

    flattened_keypoints = list(itertools.chain.from_iterable(copied_keypoints))
    max_value = max(map(abs, flattened_keypoints))

    normalized_keypoints = [value / max_value for value in flattened_keypoints]

    return normalized_keypoints

In [5]:
def write_keypoints_to_csv(label, keypoints):
    csv_file = 'hand_keypoints.csv'
    with open(csv_file, 'a', newline="") as file:
        writer = csv.writer(file)
        writer.writerow([label] + keypoints)

In [6]:
gestures =  [str(i) for i in range(1, 10)] + list(string.ascii_uppercase)

In [7]:
img_folder = 'Indian/'
image_paths = [f'{img_folder}{gesture}/{num}.jpg' for gesture in gestures for num in range(1199)]

In [8]:
with mp_hand_detection.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hand_tracker:
  
    for img_index, img_path in enumerate(image_paths):
        img = cv2.flip(cv2.imread(img_path), 1)
        results = hand_tracker.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        if not results.multi_hand_landmarks:
            continue
        
        for hand_landmarks in results.multi_hand_landmarks:
            keypoints = extract_keypoints(img, hand_landmarks)
            normalized_keypoints = normalize_keypoints(keypoints)
            write_keypoints_to_csv(img_path.split('/')[-2], normalized_keypoints)

C:\Users\91993\Desktop\ML ITER SELF LEARN\rnn\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '

KeyboardInterrupt

